In [10]:
import numpy as np
from PIL import Image
from scipy.special import j1
from scipy.ndimage import zoom

def airy_disk(r, wavelength, hole_radius, z):
    k = 2 * np.pi / wavelength
    return (2 * j1(k * hole_radius * r / z) / (k * hole_radius * r / z)) ** 2

def calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius, wavelength=550e-9):
    # 将图像转换为灰度并归一化亮度
    image = image.convert('L')
    brightness = np.array(image)*2000

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 对输入图像进行插值,使其尺寸与成像屏幕相匹配
    brightness_resized = zoom(brightness, magnification)

    # 创建一个从图像中心开始的径向距离网格
    x = np.linspace(-imaged_size[0] / 2, imaged_size[0] / 2, imaged_size[0])
    y = np.linspace(-imaged_size[1] / 2, imaged_size[1] / 2, imaged_size[1])
    xx, yy = np.meshgrid(x, y)
    r = np.sqrt(xx ** 2 + yy ** 2)

    # 计算艾里斑图案
    airy = airy_disk(r, wavelength, hole_radius, screen_distance)

    # 将艾里斑图案与插值后的输入图像亮度进行卷积
    imaged_brightness = np.fft.ifft2(np.fft.fft2(brightness_resized) * np.fft.fft2(airy)).real

    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
output_path = 'output_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 0.1  # 图像到小孔的距离(以米为单位)
screen_distance = 0.5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()
imaged_image.save(output_path, 'JPEG')

In [11]:
import numpy as np
from PIL import Image
from scipy.special import j1
from scipy.ndimage import zoom

def airy_disk(r, wavelength, hole_radius, z):
    k = 2 * np.pi / wavelength
    return (2 * j1(k * hole_radius * r / z) / (k * hole_radius * r / z)) ** 2

def calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius, wavelength=550e-9):
    # 将图像转换为灰度并归一化亮度
    image = image.convert('L')
    brightness = np.array(image) / 255.0

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 对输入图像进行插值,使其尺寸与成像屏幕相匹配
    brightness_resized = zoom(brightness, magnification)

    # 创建一个从图像中心开始的径向距离网格
    x = np.linspace(-imaged_size[0] / 2, imaged_size[0] / 2, imaged_size[0])
    y = np.linspace(-imaged_size[1] / 2, imaged_size[1] / 2, imaged_size[1])
    xx, yy = np.meshgrid(x, y)
    r = np.sqrt(xx ** 2 + yy ** 2)

    # 计算艾里斑图案
    airy = airy_disk(r, wavelength, hole_radius, screen_distance)

    # 将艾里斑图案与插值后的输入图像亮度进行卷积
    imaged_brightness = np.fft.fftshift(np.fft.ifft2(np.fft.fft2(brightness_resized) * np.fft.fft2(np.fft.fftshift(airy))).real)

    # 归一化亮度到 [0, 1] 范围内
    imaged_brightness = (imaged_brightness - np.min(imaged_brightness)) / (np.max(imaged_brightness) - np.min(imaged_brightness))

    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
output_path = 'output_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 0.1  # 图像到小孔的距离(以米为单位)
screen_distance = 0.5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()
imaged_image.save(output_path, 'JPEG')

In [2]:
import numpy as np
from PIL import Image
from scipy.special import j1
from scipy.ndimage import zoom

def airy_disk(r, wavelength, hole_radius, z):
    k = 2 * np.pi / wavelength
    return (2 * j1(k * hole_radius * r / z) / (k * hole_radius * r / z)) ** 2

def calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius, wavelength=550e-9):
    # 将图像转换为浮点数类型并归一化亮度
    brightness = np.array(image) / 255.0

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 对输入图像进行插值,使其尺寸与成像屏幕相匹配
    brightness_resized = zoom(brightness, (magnification, magnification, 1))

    # 创建一个从图像中心开始的径向距离网格
    x = np.linspace(-imaged_size[0] / 2, imaged_size[0] / 2, imaged_size[0])
    y = np.linspace(-imaged_size[1] / 2, imaged_size[1] / 2, imaged_size[1])
    xx, yy = np.meshgrid(x, y)
    r = np.sqrt(xx ** 2 + yy ** 2)

    # 计算艾里斑图案
    airy = airy_disk(r, wavelength, hole_radius, screen_distance)

    # 对每个颜色通道应用卷积
    imaged_brightness = np.zeros_like(brightness_resized)
    for i in range(3):
        imaged_brightness[:, :, i] = np.fft.fftshift(np.fft.ifft2(np.fft.fft2(brightness_resized[:, :, i]) * np.fft.fft2(np.fft.fftshift(airy))).real)

    # 归一化亮度到 [0, 1] 范围内
    imaged_brightness = (imaged_brightness - np.min(imaged_brightness)) / (np.max(imaged_brightness) - np.min(imaged_brightness))

    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
output_path = 'output_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 5  # 图像到小孔的距离(以米为单位)
screen_distance = 5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()
imaged_image.save(output_path, 'JPEG')

In [19]:
import numpy as np
from PIL import Image

def calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius,image_size):
    # 将图像转换为灰度并归一化亮度
    image = image.convert('L')
    brightness = np.array(image) / 255.0

    #定义物平面网格
    x_i=np.linspace(-image_size[0]/2,image_size[0]/2,image_size[0])
    y_i=np.linspace(-image_size[1]/2,image_size[1]/2,image_size[1])

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 创建一个从图像中心开始的径向距离网格
    x = np.linspace(-imaged_size[0] / 2, imaged_size[0] / 2, imaged_size[0])
    y = np.linspace(-imaged_size[1] / 2, imaged_size[1] / 2, imaged_size[1])
    xx, yy = np.meshgrid(x, y)
    r = np.sqrt(xx ** 2 + yy ** 2)

    # 计算余弦发光体的发光强度
    cos_emission = np.cos(np.arctan2(r / magnification, hole_distance))

    # 计算成像亮度
    imaged_brightness = np.zeros_like(brightness)
    for i in range(imaged_size[0]):
        for j in range(imaged_size[1]):
            for m in range(image_size[0]):
                for n in range(image_size[1]):
                    lens_position=[i*(hole_distance/(hole_distance+screen_distance))+m*(screen_distance/(hole_distance+screen_distance)),j*(hole_distance/(hole_distance+screen_distance))+n*(screen_distance/(hole_distance+screen_distance))]
                    if np.sqrt(lens_position[0]**2+lens_position[1]**2)<=hole_radius:
                        imaged_brightness[i,j]+=brightness[m,n]*np.cos(np.arctan2(r[i,j]/magnification,hole_distance))/((r[i,j]**2+(screen_distance)**2)*(1+magnification)**2)
    # 归一化成像亮度
    imaged_brightness = (imaged_brightness - imaged_brightness.min()) / (imaged_brightness.max() - imaged_brightness.min())

    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 0.1  # 图像到小孔的距离(以米为单位)
screen_distance = 0.5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius,image_size)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()

In [ ]:
import numpy as np
from PIL import Image

def calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius,image_size):
    # 将图像转换为灰度并归一化亮度
    image = image.convert('L')
    brightness = np.array(image) / 255.0

    #定义物平面网格
    x_i=np.linspace(-image_size[0]/2,image_size[0]/2,image_size[0])
    y_i=np.linspace(-image_size[1]/2,image_size[1]/2,image_size[1])

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 创建一个从图像中心开始的径向距离网格
    x = np.linspace(-imaged_size[0] / 2, imaged_size[0] / 2, imaged_size[0])
    y = np.linspace(-imaged_size[1] / 2, imaged_size[1] / 2, imaged_size[1])
    xx, yy = np.meshgrid(x, y)
    r = np.sqrt(xx ** 2 + yy ** 2)

    # 计算余弦发光体的发光强度
    cos_emission = np.cos(np.arctan2(r / magnification, hole_distance))

    # 计算成像亮度
    imaged_brightness = np.zeros_like(brightness)
    for i in range(imaged_size[0]):
        for j in range(imaged_size[1]):
            for m in range(image_size[0]):
                for n in range(image_size[1]):
                    lens_position=[i*(hole_distance/(hole_distance+screen_distance))+m*(screen_distance/(hole_distance+screen_distance)),j*(hole_distance/(hole_distance+screen_distance))+n*(screen_distance/(hole_distance+screen_distance))]
                    if lens_position[1]<=hole_radius & lens_position[0]<=hole_radius:
                        imaged_brightness[i,j]+=brightness[m,n]*np.cos(np.arctan2(r[i,j]/magnification,hole_distance))/((r[i,j]**2+(screen_distance)**2)*(1+magnification)**2)
    # 归一化成像亮度
    imaged_brightness = (imaged_brightness - imaged_brightness.min()) / (imaged_brightness.max() - imaged_brightness.min())

    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 0.1  # 图像到小孔的距离(以米为单位)
screen_distance = 0.5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius,image_size)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()

In [2]:
import numpy as np
from PIL import Image

def calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius, image_size):
    # 将图像转换为灰度并归一化亮度
    image = image.convert('L')
    brightness = np.array(image) / 255.0

    # 定义物平面网格
    x_i = np.linspace(-image_size[0] / 2, image_size[0] / 2, image_size[0])
    y_i = np.linspace(-image_size[1] / 2, image_size[1] / 2, image_size[1])

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 创建一个从图像中心开始的径向距离网格
    x = np.linspace(-imaged_size[0] / 2, imaged_size[0] / 2, imaged_size[0])
    y = np.linspace(-imaged_size[1] / 2, imaged_size[1] / 2, imaged_size[1])
    xx, yy = np.meshgrid(x, y)
    r = np.sqrt(xx ** 2 + yy ** 2)

    # 计算余弦发光体的发光强度
    cos_emission = np.cos(np.arctan2(r / magnification, hole_distance))

    # 计算成像亮度
    imaged_brightness = np.zeros(imaged_size)
    for i in range(imaged_size[0]):
        for j in range(imaged_size[1]):
            lens_position = [
                i * (hole_distance / (hole_distance + screen_distance)),
                j * (hole_distance / (hole_distance + screen_distance))
            ]
            if np.abs(lens_position[0]) <= hole_radius and np.abs(lens_position[1]) <= hole_radius:
                m = int((lens_position[0] + hole_radius) * image_size[0] / (2 * hole_radius))
                n = int((lens_position[1] + hole_radius) * image_size[1] / (2 * hole_radius))
                imaged_brightness[i, j] = brightness[m, n] * cos_emission[i, j] / ((r[i, j] ** 2 + screen_distance ** 2) * (1 + magnification) ** 2)

    # 归一化成像亮度
    imaged_brightness = (imaged_brightness - imaged_brightness.min()) / (imaged_brightness.max() - imaged_brightness.min())
    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 0.1  # 图像到小孔的距离(以米为单位)
screen_distance = 0.5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius, image_size)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()

In [3]:
import numpy as np
from PIL import Image

def calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius, image_size):
    # 将图像转换为灰度并归一化亮度
    image = image.convert('L')
    brightness = np.array(image) / 255.0

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 创建一个从图像中心开始的径向距离网格
    x = np.linspace(-imaged_size[0] / 2, imaged_size[0] / 2, imaged_size[0])
    y = np.linspace(-imaged_size[1] / 2, imaged_size[1] / 2, imaged_size[1])
    xx, yy = np.meshgrid(x, y)
    r = np.sqrt(xx ** 2 + yy ** 2)

    # 计算余弦发光体的发光强度
    cos_emission = np.cos(np.arctan2(r, screen_distance))

    # 计算成像亮度
    imaged_brightness = np.zeros(imaged_size)
    for i in range(imaged_size[0]):
        for j in range(imaged_size[1]):
            lens_position = [
                (i - imaged_size[0] / 2) * (hole_distance / screen_distance),
                (j - imaged_size[1] / 2) * (hole_distance / screen_distance)
            ]
            if np.abs(lens_position[0]) <= hole_radius and np.abs(lens_position[1]) <= hole_radius:
                m = int((lens_position[0] / hole_radius + 0.5) * image_size[0])
                n = int((lens_position[1] / hole_radius + 0.5) * image_size[1])
                imaged_brightness[i, j] = brightness[m, n] * cos_emission[i, j] / (r[i, j] ** 2 + screen_distance ** 2)

    # 归一化成像亮度
    imaged_brightness = (imaged_brightness - imaged_brightness.min()) / (imaged_brightness.max() - imaged_brightness.min())
    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 0.1  # 图像到小孔的距离(以米为单位)
screen_distance = 0.5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = calculate_imaged_brightness(image, hole_distance, screen_distance, hole_radius, image_size)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()

In [5]:
import numpy as np
from PIL import Image

def ray_tracing(image, hole_distance, screen_distance, hole_radius, image_size, num_rays):
    # 将图像转换为灰度并归一化亮度
    image = image.convert('L')
    brightness = np.array(image) / 255.0

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 创建成像亮度数组
    imaged_brightness = np.zeros(imaged_size)

    # 生成随机光线方向
    theta = np.random.uniform(0, 2 * np.pi, num_rays)
    phi = np.arcsin(np.random.uniform(-hole_radius, hole_radius, num_rays) / hole_distance)

    # 光线追踪
    for i in range(num_rays):
        # 计算光线与物平面的交点
        x_obj = hole_distance * np.tan(phi[i]) * np.cos(theta[i])
        y_obj = hole_distance * np.tan(phi[i]) * np.sin(theta[i])

        # 检查交点是否在图像范围内
        if abs(x_obj) <= image_size[0] / 2 and abs(y_obj) <= image_size[1] / 2:
            # 计算交点在图像上的坐标
            m = int((x_obj + image_size[0] / 2) * image.width / image_size[0])
            n = int((y_obj + image_size[1] / 2) * image.height / image_size[1])

            # 计算光线与成像屏幕的交点
            x_screen = screen_distance * np.tan(phi[i]) * np.cos(theta[i])
            y_screen = screen_distance * np.tan(phi[i]) * np.sin(theta[i])

            # 计算交点在成像屏幕上的坐标
            p = int((x_screen + imaged_size[0] / 2) / magnification)
            q = int((y_screen + imaged_size[1] / 2) / magnification)

            # 检查坐标是否在成像屏幕范围内
            if 0 <= p < imaged_size[0] and 0 <= q < imaged_size[1]:
                # 累加亮度值
                imaged_brightness[p, q] += brightness[m, n] / num_rays

    # 归一化成像亮度
    imaged_brightness = (imaged_brightness - imaged_brightness.min()) / (imaged_brightness.max() - imaged_brightness.min())
    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 0.1  # 图像到小孔的距离(以米为单位)
screen_distance = 0.5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)
num_rays = 1000000  # 光线数量

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = ray_tracing(image, hole_distance, screen_distance, hole_radius, image_size, num_rays)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()

In [ ]:
import numpy as np
from PIL import Image

def ray_tracing(image, hole_distance, screen_distance, hole_radius, image_size, num_rays):
    # 将图像转换为灰度并归一化亮度
    image = image.convert('L')
    brightness = np.array(image) / 255.0

    # 计算放大倍数
    magnification = screen_distance / hole_distance

    # 计算成像屏幕的尺寸
    imaged_size = (int(image.width * magnification), int(image.height * magnification))

    # 创建成像亮度数组
    imaged_brightness = np.zeros(imaged_size)

    # 生成随机光线方向
    theta = np.random.uniform(0, 2 * np.pi, num_rays)
    phi = np.arcsin(np.random.uniform(-hole_radius, hole_radius, num_rays) / np.sqrt(hole_distance**2 + hole_radius**2))

    # 光线追踪
    for i in range(num_rays):
        # 计算光线与物平面的交点
        x_obj = hole_distance * np.tan(phi[i]) * np.cos(theta[i])
        y_obj = hole_distance * np.tan(phi[i]) * np.sin(theta[i])

        # 检查交点是否在图像范围内
        if abs(x_obj) <= image_size[0] / 2 and abs(y_obj) <= image_size[1] / 2:
            # 计算交点在图像上的坐标
            m = int((x_obj + image_size[0] / 2) * image.width / image_size[0])
            n = int((y_obj + image_size[1] / 2) * image.height / image_size[1])

            # 计算光线与成像屏幕的交点
            x_screen = screen_distance * np.tan(phi[i]) * np.cos(theta[i])
            y_screen = screen_distance * np.tan(phi[i]) * np.sin(theta[i])

            # 计算交点在成像屏幕上的坐标
            p = int((x_screen / magnification + imaged_size[0] / 2))
            q = int((y_screen / magnification + imaged_size[1] / 2))

            # 检查坐标是否在成像屏幕范围内
            if 0 <= p < imaged_size[0] and 0 <= q < imaged_size[1]:
                # 累加亮度值
                imaged_brightness[q, p] += brightness[n, m] / num_rays

    # 归一化成像亮度
    imaged_brightness = (imaged_brightness - imaged_brightness.min()) / (imaged_brightness.max() - imaged_brightness.min())
    return imaged_brightness

# 输入值
image_path = 'test_image.jpg'
image_size = (500, 500)  # 宽度,高度
hole_distance = 0.1  # 图像到小孔的距离(以米为单位)
screen_distance = 0.5  # 小孔到成像屏幕的距离(以米为单位)
hole_radius = 0.0001  # 小孔半径(以米为单位)
num_rays = 1000000  # 光线数量

# 读取输入图像
image = Image.open(image_path).resize(image_size)

# 计算成像亮度
imaged_brightness = ray_tracing(image, hole_distance, screen_distance, hole_radius, image_size, num_rays)

# 显示成像亮度
imaged_image = Image.fromarray((imaged_brightness * 255).astype(np.uint8))
imaged_image.show()

In [6]:
imaged_image.save('output_image.jpg', 'JPEG')